In [ ]:
# Cell 1: Import necessary libraries for notebook interaction
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

# Optional: Configure pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.3f}'.format) # Format floats to 3 decimal places

In [1]:
# Cell 2: Run the processing script
print("Running experiment processing script (using %run)...")
%run metrics.py
print("Script execution finished (using %run).")

Running experiment processing script (using %run)...
--- Metrics Generator Script Starting (v3 - Full) ---
Using device: cuda
--- Initializing... ---
--- Pass 1: Processing Baselines to establish FLOPs/Params references ---

--- Baseline references populated: {} ---

--- Pass 2: Processing ALL other (non-baseline) experiments ---

  Processing experiment: resnet18pretrained_distilled_quant_kmeans_256clusters_post from category 'combined_distilled_quantized'
      WARNING (resnet18pretrained_distilled_quant_kmeans_256clusters_post): Baseline FLOPs/Params for ResNet18 not found for quantized model.

  Processing experiment: resnet18pretrained_distilled_quant_ptq_int8_perchannel_post from category 'combined_distilled_quantized'
      INFO (resnet18pretrained_distilled_quant_ptq_int8_perchannel_post): Attempting to set quantized engine to 'x86'.
      ERROR (resnet18pretrained_distilled_quant_ptq_int8_perchannel_post): General torch.load also failed: The following operation failed in the T

c:\Users\Saad\miniconda3\envs\dl-opt\Lib\site-packages\torch\serialization.py:1328: UserWarning: 'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)
  warnings.warn(



  Processing experiment: resnet50_to_resnet18scratch_kd from category 'knowledge_distillation'

  Processing experiment: resnet50_prune_nm24_ft from category 'pruning_nm_sparsity'

  Processing experiment: resnet50_prune_struct_it_l1filter_stage1_approx_sp50_ft from category 'pruning_structured_iterative'

  Processing experiment: resnet50_prune_struct_it_l1filter_stage2_approx_sp75_ft from category 'pruning_structured_iterative'

  Processing experiment: resnet50_prune_struct_it_l1filter_stage3_approx_sp90_ft from category 'pruning_structured_iterative'

  Processing experiment: resnet50_prune_struct_os_l1filter_fp30_ft from category 'pruning_structured_oneshot'

  Processing experiment: resnet50_prune_struct_os_l1filter_fp55_ft from category 'pruning_structured_oneshot'

  Processing experiment: resnet50_prune_struct_os_l1filter_fp70_ft from category 'pruning_structured_oneshot'

  Processing experiment: resnet50_prune_unstruct_it_l1_stage1_sp50_ft from category 'pruning_unstructure

In [ ]:
# Cell 3: Load and display the generated CSV
csv_file_path = "model_optimization_summary_uniform_eval.csv"

if os.path.exists(csv_file_path):
    print(f"\nLoading data from {csv_file_path}...")
    df_summary = pd.read_csv(csv_file_path)

    print("\n--- DataFrame Head ---")
    display(df_summary.head())

    print("\n--- DataFrame Info ---")
    df_summary.info()

    print("\n--- DataFrame Description (Numerical Columns) ---")
    # Select numeric columns for describe, handling potential non-numeric inference times
    numeric_cols = df_summary.select_dtypes(include=np.number).columns
    display(df_summary[numeric_cols].describe())
    
    # Identify columns that were expected to be numeric but might have errors
    potential_numeric_issues = []
    for col in ['Final_Val_Accuracy', 'Model_Size_MB_Disk', 'Inference_Time_ms_CPU (Batch 1)']:
         if df_summary[col].dtype == 'object': # if it's not numeric
            try:
                pd.to_numeric(df_summary[col]) # test conversion
            except ValueError:
                potential_numeric_issues.append(col)
    if potential_numeric_issues:
        print(f"\nWarning: Columns {potential_numeric_issues} contain non-numeric 'Measurement Error' or 'N/A' values.")
        print("Consider cleaning these (e.g., replacing with NaN) for numerical analysis/plotting if needed.")
        # Example: df_summary['Inference_Time_ms_CPU (Batch 1)'] = pd.to_numeric(df_summary['Inference_Time_ms_CPU (Batch 1)'], errors='coerce')


else:
    print(f"Error: {csv_file_path} not found. Ensure the script ran successfully.")


In [ ]:
# Cell 4: Example Visualizations (add more as needed for your presentation)
if 'df_summary' in locals() and not df_summary.empty:
    print("\n--- Example Visualizations ---")
    
    # Convert relevant columns to numeric, coercing errors for plotting
    plot_df = df_summary.copy()
    cols_to_numeric = [
        'Final_Val_Accuracy', 'Model_Size_MB_Disk', 
        'Inference_Time_ms_CPU (Batch 1)', 'Achieved_Sparsity_Percent',
        'Accuracy_Change_vs_Baseline_pp', 'Model_Size_Reduction_vs_Baseline_Percent',
        'Inference_Speedup_vs_Baseline_CPU'
    ]
    for col in cols_to_numeric:
        if col in plot_df.columns:
             plot_df[col] = pd.to_numeric(plot_df[col], errors='coerce')

    # Filter out baselines for some plots to see effect of optimizations
    optimized_df = plot_df[plot_df['Optimization_Category'] != 'Baseline'].copy()

    # 1. Accuracy vs. Model Size
    plt.figure(figsize=(10, 6))
    sns.scatterplot(data=optimized_df, x='Model_Size_MB_Disk', y='Final_Val_Accuracy', hue='Optimization_Category', size='Inference_Time_ms_CPU (Batch 1)', style='Base_Model_Arch')
    plt.title('Final Validation Accuracy vs. Model Size (MB on Disk)')
    plt.xlabel('Model Size (MB)')
    plt.ylabel('Final Validation Accuracy')
    plt.grid(True)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()

    # 2. Accuracy Retention vs. Model Size Reduction (for optimized models)
    plt.figure(figsize=(10, 6))
    sns.scatterplot(data=optimized_df, x='Model_Size_Reduction_vs_Baseline_Percent', y='Accuracy_Retention_Percent', hue='Specific_Technique', style='Base_Model_Arch')
    plt.title('Accuracy Retention vs. Model Size Reduction (Compared to Baseline)')
    plt.xlabel('Model Size Reduction (%)')
    plt.ylabel('Accuracy Retention (%)')
    plt.axhline(100, color='gray', linestyle='--', label='100% Retention')
    plt.axvline(0, color='gray', linestyle='--')
    plt.grid(True)
    plt.legend(title='Specific Technique', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()

    # 3. Inference Speedup vs. Accuracy Change (CPU)
    plt.figure(figsize=(10, 6))
    sns.scatterplot(data=optimized_df, x='Inference_Speedup_vs_Baseline_CPU', y='Accuracy_Change_vs_Baseline_pp', hue='Optimization_Category', style='Base_Model_Arch')
    plt.title('CPU Inference Speedup vs. Accuracy Change (pp from Baseline)')
    plt.xlabel('Inference Speedup (Factor over Baseline)')
    plt.ylabel('Accuracy Change (Percentage Points)')
    plt.axhline(0, color='gray', linestyle='--', label='No Accuracy Change')
    plt.axvline(1, color='gray', linestyle='--', label='No Speedup')
    plt.grid(True)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()
    
    # 4. Bar plot of Final Accuracy by Specific Technique (Top N techniques)
    if not optimized_df.empty:
        plt.figure(figsize=(12, 7))
        # Sort by accuracy and take top N or all
        top_techniques = optimized_df.groupby('Specific_Technique')['Final_Val_Accuracy'].mean().sort_values(ascending=False)
        sns.barplot(x=top_techniques.index, y=top_techniques.values, palette="viridis")
        plt.xticks(rotation=45, ha="right")
        plt.title('Average Final Validation Accuracy by Specific Technique')
        plt.ylabel('Average Final Validation Accuracy')
        plt.xlabel('Specific Technique')
        plt.tight_layout()
        plt.show()

else:
    print("DataFrame could not be loaded or is empty, skipping visualizations.")